In [1]:
pip install feedparser

In [19]:
import pandas as pd
import feedparser
from tqdm import tqdm

In [3]:
feeds = pd.read_csv('/content/Crypto feeds.csv')
urls = list(feeds.Feeds)
feeds = [feedparser.parse(url)['entries'] for url in urls]

убираю пустые фиды

In [4]:
non_zero_feeds = []
for i in range(len(feeds)):
  if len(feeds[i]) != 0:
    non_zero_feeds.append(feeds[i])
  else:
    print(pd.read_csv('/content/Crypto feeds.csv').iloc[i].Feeds)

https://blog.bitpanda.com/feed
https://coinatmradar.com/blog/feed/?x=1
https://www.coin.space/feed/
https://blog.coinspectator.com/feed/
https://belfrics.com/blog/feed/
https://www.hegion.com/feed
https://bitcoin-millionaire.com/feed/
https://blog.flitpay.in/feed/
https://paperblockchain.com/feed/
http://bitcoinmxn.com/feed
https://blog.spectrocoin.com/en/feed/
https://oilblockchain.news/feed/
https://coincurrencynews.com/feed/
https://www.goldsilveranalyst.com/feed
https://medium.com/feed/@satoshiknotabolo
https://blog.zebpay.com/feed
https://bitcoinspakistan.com/feed/
https://blog.feedspot.com/feed/


ищу фиды, в которых для каждой новости не указана дата

In [5]:

for i in range(len(non_zero_feeds)):
  if ('published' not in non_zero_feeds[i][1].keys()) and ('pubDate' not in non_zero_feeds[i][1].keys()):
    print(i)


39


In [6]:
needed_feeds = non_zero_feeds[:39] + non_zero_feeds[40:]

преобразую в датафрейм

In [18]:
news = pd.DataFrame(columns = {'title':[], 'date_published':[]})

for k in needed_feeds:
  for j in range(len(k)):
    if ('published' in k[j].keys()):
      unit = pd.DataFrame({'title': [k[j].title], 'date_published': [k[j].published]})
    elif ('pubDate' in k[j].keys()):
      unit = pd.DataFrame({'title': [k[j].title], 'date_published': [k[j].pubDate]})
    news = pd.concat([news, unit], axis = 0)

In [24]:
news

,title,date_published
0,Algorand aims to convert network transaction f...,"Thu, 21 Apr 2022 10:15:31 +0100"
0,GBTC premium nears 2022 high as SEC faces call...,"Thu, 21 Apr 2022 09:08:01 +0100"
0,Binance limits services in Russia due to the E...,"Thu, 21 Apr 2022 09:07:32 +0100"
0,US investors realized 6x more crypto gains in ...,"Thu, 21 Apr 2022 06:42:17 +0100"
0,German banking giant Commerzbank applies for c...,"Thu, 21 Apr 2022 05:41:33 +0100"
...,...,...
0,BTC Media Announces Comedy Hour As Part Of Sou...,"Mon, 04 Apr 2022 11:57:58 +0000"
0,Defactor Is First To Bring DeFi Finance Servic...,"Sun, 03 Apr 2022 17:52:21 +0000"
0,Defactor Announces Partnership With Amplify Pr...,"Fri, 01 Apr 2022 09:56:04 +0000"
0,SparkWorld* announces partnership with Islande...,"Thu, 31 Mar 2022 14:57:34 +0000"


In [25]:
news.date_published.min()

'Fri, 01 Apr 2022 07:53:05 +0000'